In [46]:
import jieba
import jieba.analyse as analyse
import numpy as np

In [130]:
class SimHash(object):
    def __init__(self, content, topK=50):
        self.topK = topK
        self.simhash = self.getSimHash(content)
    
    def getSimHash(self, content):
        seg = jieba.cut(content)
#         jieba.analyse.set_stop_words('stopword.txt')
        #topk words and it's tf/idf 
        keyWords = jieba.analyse.extract_tags(
            '|'.join(seg), topK=self.topK, withWeight=True, allowPOS=())
#         print(keyWords)
        word_list = []
        for feature, weight in keyWords:
            feature = self.string_hash(feature)
            temp = []
            for i in feature:
                if i == '1':
                    temp.append(weight)
                else:
                    temp.append(-weight)
            word_list.append(temp)
        hashSum = np.sum(np.array(word_list), axis=0)
        simhash = ''
        for code in hashSum:
            if code > 0:
                simhash += '1'
            else:
                simhash += '0'
        return simhash
    
    def string_hash(self,source):
        if source == "":
            return 0
        else:
            x = ord(source[0]) << 7
            m = 1000003
            mask = 2 ** 128 - 1
            for c in source:
                x = ((x * m) ^ ord(c)) & mask
            x ^= len(source)
            if x == -1:
                x = -2
            x = bin(x).replace('0b', '').zfill(64)[-64:]
#             print(source,x)
            return x
 


In [131]:
def hamDis(l1, l2):
    #异或
    lxor = int(l1,2) ^ int(l2,2)
    c = 0
    #计算异或结果1的个数
    while(lxor):
        lxor &= lxor-1
        c += 1
    return c

In [132]:
a = SimHash('''机器学习(Machine Learning, ML)是一门多领域交叉学科，涉及概率论、统计学、逼近论、凸分析、算法复杂度理论等多门学科。
专门研究计算机怎样模拟或实现人类的学习行为，以获取新的知识或技能，重新组织已有的知识结构使之不断改善自身的性能。
它是人工智能的核心，是使计算机具有智能的根本途径，其应用遍及人工智能的各个领域，它主要使用归纳、综合而不是演绎。''')
b = SimHash('''机器学习有下面几种定义： “机器学习是一门人工智能的科学，该领域的主要研究对象是人工智能，特别是如何在经验学习中
改善具体算法的性能”。 “机器学习是对能通过经验自动改进的计算机算法的研究”。 “机器学习是用数据或以往的经验，以此优化计算机程序的性能
标准。” 一种''')

In [133]:
hamDis(a.simhash, b.simhash)

18

In [134]:
con1 = '''本文思路

Hbase是什么
Hbase的优劣
Hbase架构
Hbase容错
Hbase使用总结

HBase是什么

基于HDFS的列式存储NoSql数据库
适合于非结构化数据存储的数据库

Hbase的特点
1.采用稀疏列式存储，节约存储空间
2.可以自动切分数据，能够水平拓展
3.可以提供高并发，低延迟的读写操作
缺点
不支持丰富的条件筛选，只能根据rowkey进行get或者scan读取
Hbase架构







Client
hbase的访问客户端，可以做一些本地缓存，来加快对hbase集群的访问，如 region的位置信息
HMaster
负责HRegionServer的负载均衡，给HRegionServier分配Region。client读取数据不经过HMaster。
HRegionServer
管理Region，负责Region的切分和合并
HRegion


Table在行的方向上分割为多个HRegion，HRegion是HBase中分布式存储和负载均衡的最小单元，即不同的HRegion可以分别在不同的HRegionServer上，但同一个HRegion是不会拆分到多个HRegionServer上的。HRegion按大小分割，每个表一般只有一个HRegion，随着数据不断插入表，HRegion不断增大，当HRegion的某个列簇达到一个阀值（默认256M）时就会分成两个新的HRegion。






Store






Region虽然是分布式存储的最小单元，但并不是存储的最小单元。Region由一个或者多个Store组成，每个store保存一个columns family；每个Strore又由一个memStore和0至多个StoreFile组成，StoreFile包含HFile；memStore存储在内存中，StoreFile存储在HDFS上(数据写入先写memStore，当memStore超过阀值（默认64MB），则会刷入以StoreFile磁盘)。

Zookeeper
zk的引入，第一防止HMaster单点故障。另外还作为存储，保存-Root- 的HRegionServer位置。
物理架构







HBase容错性
Master容错：Zookeeper重新选择一个新的Master

无Master过程中，数据读取仍照常进行；
无master过程中，region切分、负载均衡等无法进行；

RegionServer容错：定时向Zookeeper汇报心跳，如果一旦时间内未出现心跳，Master将该RegionServer上的Region重新分配到其他RegionServer上，失效服务器上“预写”日志由主服务器进行分割并派送给新的RegionServer
Zookeeper容错：
Zookeeper是一个可靠地服务，一般配置3或5个Zookeeper实例
HBase写入







每个HRegionServer中都有一个HLog对象，HLog是一个实现Write Ahead Log的类，在每次用户操作写入MemStore的同时，也会写一份数据到HLog文件中（HLog文件格式见后续），HLog文件定期会滚动出新的，并删除旧的文件（已持久化到StoreFile中的数据）。当HRegionServer意外终止后，HMaster会通过Zookeeper感知到，HMaster首先会处理遗留的  HLog文件，将其中不同Region的Log数据进行拆分，分别放到相应region的目录下，然后再将失效的region重新分配，领取 到这些region的HRegionServer在Load Region的过程中，会发现有历史HLog需要处理，因此会Replay HLog中的数据到MemStore中，然后flush到StoreFiles，完成数据恢复

HBase读取
读数据的关键是定位Region所在的RegionServer，数据的RegionServer如果被定位过，会缓存到client端。
HBase里有两个特殊表分别是 -ROOT- 和 .META.
-ROOT-
表包含.META.表所在的region列表，该Region永不拆分，只有一个。
.META.
表包含所有的用户表对应的region的位置信息。
定位RegionServer
ZooKeeper--> -ROOT-(单Region)--> .META.--> 用户表
ZK的结构如下
整个过程如下








Hbase使用总结
1.rowKey 必须尽量的分散，避免数据倾斜到单个Region节点。

自增id开头的，一定要 reverse 反转一下
罗列出查询场景，看看rowkey是否能满足查询需要
scan查询，默认一次只返回100条数据，如果需要返回的更多，需要设置cache参数
mutliGet注意控制 rowkey数量，针对rt设置一个合理的查询上限
'''

In [135]:
con2 = '''HBase是一个构建在HDFS上的分布式列存储系统；

 HBase是基于Google BigTable模型开发的，典型的key/value系统；

 HBase是Apache Hadoop生态系统中的重要一员，主要用于海量结构化数据存储；

 从逻辑上讲，HBase将数据按照表、行和列进行存储。

 与hadoop一样，Hbase目标主要依靠横向扩展，通过不断增加廉价的商用服务器，来增加计算和存储能力。

Hbase表的特点

 大：一个表可以有数十亿行，上百万列；

 无模式：每行都有一个可排序的主键和任意多的列，列可以根据需要动态的增加，同一张表中不同的行可以有截然不同的列；

 面向列：面向列（族）的存储和权限控制，列（族）独立检索；

 稀疏：空（null）列并不占用存储空间，表可以设计的非常稀疏；

 数据多版本：每个单元中的数据可以有多个版本，默认情况下版本号自动分配，是单元格插入时的时间戳；

数据类型单一：Hbase中的数据都是字符串，没有类型。

Hbase数据模型
Hbase逻辑视图

 注意上图中的英文说明
 

Hbase基本概念
 RowKey：是Byte array，是表中每条记录的“主键”，方便快速查找，Rowkey的设计非常重要。
 Column Family：列族，拥有一个名称(string)，包含一个或者多个相关列
 Column：属于某一个columnfamily，familyName:columnName，每条记录可动态添加
 Version Number：类型为Long，默认值是系统时间戳，可由用户自定义
 Value(Cell)：Byte array
 

Hbase物理模型
 每个column family存储在HDFS上的一个单独文件中，空值不会被保存。

 Key 和 Version number在每个 column family中均有一份；

 HBase 为每个值维护了多级索引，即：<key, column family, column name, timestamp>

物理存储:
 1、Table中所有行都按照row key的字典序排列；

2、Table在行的方向上分割为多个Region；

3、Region按大小分割的，每个表开始只有一个region，随着数据增多，region不断增大，当增大到一个阀值的时候，region就会等分会两个新的region，之后会有越来越多的region；
 

4、Region是Hbase中分布式存储和负载均衡的最小单元，不同Region分布到不同RegionServer上。
 


5、Region虽然是分布式存储的最小单元，但并不是存储的最小单元。Region由一个或者多个Store组成，每个store保存一个columns family；每个Strore又由一个memStore和0至多个StoreFile组成，StoreFile包含HFile；memStore存储在内存中，StoreFile存储在HDFS上。

 


HBase架构及基本组件

Hbase基本组件说明：
 Client
 包含访问HBase的接口，并维护cache来加快对HBase的访问，比如region的位置信息
 Master
 为Region server分配region
 负责Region server的负载均衡
 发现失效的Region server并重新分配其上的region
 管理用户对table的增删改查操作
 Region Server
 Regionserver维护region，处理对这些region的IO请求
 Regionserver负责切分在运行过程中变得过大的region
 Zookeeper作用
 通过选举，保证任何时候，集群中只有一个master，Master与RegionServers 启动时会向ZooKeeper注册
 存贮所有Region的寻址入口
 实时监控Region server的上线和下线信息。并实时通知给Master
 存储HBase的schema和table元数据
 默认情况下，HBase 管理ZooKeeper 实例，比如， 启动或者停止ZooKeeper
 
 Zookeeper的引入使得Master不再是单点故障
 


 
 
 

Write-Ahead-Log（WAL）

该机制用于数据的容错和恢复：

每个HRegionServer中都有一个HLog对象，HLog是一个实现Write Ahead Log的类，在每次用户操作写入MemStore的同时，也会写一份数据到HLog文件中（HLog文件格式见后续），HLog文件定期会滚动出新的，并删除旧的文件（已持久化到StoreFile中的数据）。当HRegionServer意外终止后，HMaster会通过Zookeeper感知到，HMaster首先会处理遗留的  HLog文件，将其中不同Region的Log数据进行拆分，分别放到相应region的目录下，然后再将失效的region重新分配，领取 到这些region的HRegionServer在Load Region的过程中，会发现有历史HLog需要处理，因此会Replay HLog中的数据到MemStore中，然后flush到StoreFiles，完成数据恢复

HBase容错性
Master容错：Zookeeper重新选择一个新的Master

 无Master过程中，数据读取仍照常进行；

 无master过程中，region切分、负载均衡等无法进行；

RegionServer容错：定时向Zookeeper汇报心跳，如果一旦时间内未出现心跳，Master将该RegionServer上的Region重新分配到其他RegionServer上，失效服务器上“预写”日志由主服务器进行分割并派送给新的RegionServer

Zookeeper容错：Zookeeper是一个可靠地服务，一般配置3或5个Zookeeper实例

Region定位流程：


寻找RegionServer
ZooKeeper--> -ROOT-(单Region)--> .META.--> 用户表

-ROOT-

表包含.META.表所在的region列表，该表只会有一个Region；

Zookeeper中记录了-ROOT-表的location。

.META.

表包含所有的用户空间region列表，以及RegionServer的服务器地址。

Hbase使用场景
storing large amounts  of data(100s of TBs)
 need high write throughput
 need efficient random access(key lookups) within large data sets
 need to scale gracefully with data
 for structured and semi-structured data
 don't need fullRDMS capabilities(cross row/cross table transaction, joins,etc.)
 

大数据量存储，大数据量高并发操作

需要对数据随机读写操作

读写访问均是非常简单的操作

Hbase与HDFS对比
两者都具有良好的容错性和扩展性，都可以扩展到成百上千个节点；

HDFS适合批处理场景

不支持数据随机查找

不适合增量数据处理

不支持数据更新'''

In [136]:
h1 = SimHash(con1)
h2 = SimHash(con2)

In [137]:
hamDis(h1.simhash, h2.simhash)

11

In [148]:
con3 = '''凤姐怎么出名的：凤姐本名叫罗玉凤，罗玉凤是重庆綦江人，因一系列雷人言论在网络上走红，被人称为“凤姐”。她自称懂诗画、会弹琴，精通古汉语，自称“9岁起博览群书，20岁达到顶峰，智商前300年后300年无人能及”。现主要研读经济类和《知音》、《故事会》等人文社科类书籍。

凤姐怎么出名的
　　征婚标准

　　本人找伴侣。一不求帅。二不求富。但求同甘苦，共患难。

　　本人对伴侣要求如下：

　　第一，必须为北京大学或清华大学硕士毕业生。必须本科硕士连读，中途无跳级，不留级，不转校。在外参加工作后再回校读书者免。

　　第二，必须为经济学专业毕业。非经济学专业毕业则必须精通经济学。或对经济学有浓厚的兴趣。

　　第三，必须具备国际视野，但是无长期定居国外甚至移民的打算。

　　第四，身高176--183左右。长得越帅越好。必须有刘德华那样的帅气、任达华那样的性感、立威廉那样的俊俏、谢霆锋那样的冷酷。

　　第五，无生育史。过往所有女友均无因自身而致的堕胎史。

　　第六，东部户籍，即江、浙、沪三地户籍或广东、天津、山东、福建、北京、东北三省和内蒙古等地户籍。

　　西南地区即四川、重庆、贵州、云南、西藏和湖南、湖北等地籍贯者不予考虑。

　　第七，年龄25--28岁左右。即06届，07届，08届，09届毕业生。有一至两年的工作经验，06级毕业生需年龄在28岁左右，09级毕业生则需聪明过人。且具备丰富的社会实践经验。就职于国家机关，国有企事业单位者不予考虑。但就职于中石油，中石化等世界顶尖型企业或银行者又比较喜欢。现自主创业者要商榷一番了。

　　本人85年旧历8月初9日生。新历生日为9月23日。身高146。平时穿高跟鞋153。体重40kg.先就读于綦江师范学校获中师文凭。

　　后连读重庆教育学院获汉语言文学专业大专文凭。懂诗画，唱歌，弹琴，刺绣等。最擅长诗歌与散文。并精通古汉语。博览群书。较为狂妄。无堕胎史，无生育史。交过几个不了了之的男朋友。具体进展却无。主要要求男方身家清白，聪慧过人。'''

In [149]:
con4 = '''曾经的网络红人，罗玉凤，美称“凤姐”，让我看看凤姐开出七大征婚条件：
第一，必须为北京大学或清华大学硕士毕业生。必须本科硕士连读，中途无跳级，不留级，不转校。在外参加工作后再回校读书者免。
第二，必须为经济学专业毕业。非经济学专业毕业则必须精通经济学。或对经济学有浓厚的兴趣。
第三，必须具备国际视野，但是无长期定居国外甚至移民的打算。第四，身高176--183左右。长得帅的比较好一些。
第五，无生育史。过往所有女友均无因自身而致的堕胎史。
第六，东部沿海户籍，即江，浙，沪三地户籍或广东，天津，山东。北京，东北三省，内蒙古等地户籍。西南地区即重庆。贵州。云南。西藏，湖南，
湖北等地籍贯者不予考虑。第七。年龄25--28岁左右。即06级，07级，08级，09级毕业生。有一至两年的工作经验，06级毕业生需年龄在28岁左右
，09级毕业生则需聪明过人。且具备丰富的社会实践经验。就职于国家机关，国有企事业单位者不愿考虑。但就职于中石油，
中石化等世界顶尖型企业或银行者又比较喜欢。现自主创业者要商榷一番了。然后，凤姐自我介绍了一番。本人85年旧历8月初9日生。
新历生日为9月23日。身高146CM ，平时穿高跟鞋153。体重40kg.先就读于綦江师范学校获中师文凭。后连读重庆教育学院获汉语言文学专业大专文凭。
懂诗画，唱歌，弹琴，刺绣等。最擅长诗歌与散文。并精通古汉语。博览群书。较为狂妄。无堕胎史，无生育史。交过几个不了了之的男朋友。
具体进展却无。主要要求男方身家清白，聪慧过人。
'''

In [150]:
hamDis(SimHash(con3).simhash, SimHash(con4).simhash)

14